In [0]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import regexp_replace

#GEO DATAFRAME
#Databricks code
#Reading the geo dataframe
file_type = "json"
infer_schema = "true"

file_location_geo = "/mnt/0a65154c50dd-mount/topics/0a65154c50dd.geo/partition=0//*.json" 
df_geo = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_geo)

df_geo.createOrReplaceTempView("geo")


#Data Cleaning 

df_geo = spark.sql("SELECT * FROM geo")
df_geo = df_geo.withColumn('coordinates',F.array(df_geo.latitude,df_geo.longitude))
df_geo = df_geo.drop("latitude","longitude")
df_geo = df_geo.withColumn("timestamp",  
                                  df_geo["timestamp"] 
                                  .cast('timestamp')) 
df_geo = df_geo.select("ind","country","coordinates","timestamp")
df_geo.show()

+-----+--------------------+--------------------+-------------------+
 ind| country| coordinates| timestamp|
+-----+--------------------+--------------------+-------------------+
 2301|Saint Vincent and...| [13.4683, 51.7244]|2020-11-14 00:25:28|
 5293|Sao Tome and Prin...|[-13.1463, -25.9649]|2019-05-31 20:49:36|
 2604| Antigua and Barbuda|[-80.8933, -104.972]|2018-12-01 09:23:35|
 2048| Antigua and Barbuda|[-89.4008, -142.186]|2017-11-20 06:33:50|
10119| Christmas Island|[-74.5431, -162.795]|2020-10-22 01:59:58|
 2060| Trinidad and Tobago| [52.4584, 68.6527]|2020-01-25 13:54:17|
 6261| Marshall Islands|[-10.3101, -109.763]|2021-11-11 16:18:45|
10337| Christmas Island| [-42.3069, -109.17]|2021-10-26 12:09:43|
 3201| Pitcairn Islands| [34.0532, -68.4946]|2019-06-25 08:31:37|
 1422| American Samoa|[-88.5252, -172.436]|2018-04-30 08:27:21|
 8312| American Samoa|[-77.9744, -106.258]|2021-04-25 15:56:29|
 1450| American Samoa|[-88.5252, -172.436]|2022-01-15 16:04:27|
 9268| American Samoa|[-88.5255, -161.644]|2020-03-16 23:13:51|
 8578| Guinea-Bissau| [34.6427, -161.397]|2018-01-26 19:27:11|
 6844| New Caledonia| [-22.6915, 5.69245]|2021-06-06 21:53:11|
 8978| Faroe Islands| [-2.01499, 3.78198]|2017-12-01 13:26:06|
 3599| Afghanistan|[-88.5478, -174.971]|2019-03-03 06:13:41|
 4988| Madagascar|[-32.1879, -35.2784]|2020-07-11 11:01:12|
 427| Isle of Man|[-66.9418, -30.0087]|2020-04-22 03:08:50|
 5758| Azerbaijan|[-79.3714, -145.242]|2018-09-12 15:08:34|
+-----+--------------------+--------------------+-------------------+
only showing top 20 rows

In [0]:
#USER DATAFRAME 

file_location_user = "/mnt/0a65154c50dd-mount/topics/0a65154c50dd.user/partition=0//*.json" 
df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_user)

df_user.createOrReplaceTempView("user")

#Data Cleaning

df_user = spark.sql("SELECT * FROM user")

col_list = ['first_name','last_name']
df_user = df_user.withColumn('user_name',concat_ws(' ',*col_list))
df_user = df_user.drop('first_name','last_name')
df_user = df_user.withColumn("date_joined",  
                                  df_user["date_joined"] 
                                  .cast('timestamp')) 
df_user = df_user.withColumn("age",  
                                  df_user["age"] 
                                  .cast('int')) 
df_user = df_user.select("ind","user_name","age","date_joined")

In [0]:
#PIN DATAFRAME

file_location_pin = "/mnt/0a65154c50dd-mount/topics/0a65154c50dd.pin/partition=0//*.json" 
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_pin)

df_pin.createOrReplaceTempView("pin")

#Data Cleaning

df_pin = spark.sql("SELECT * FROM pin")
df_pin = df_pin.withColumnRenamed('index', 'ind')
df_pin = df_pin.withColumn('follower_count', regexp_replace('follower_count', 'k', '000'))
df_pin = df_pin.withColumn('follower_count', regexp_replace('follower_count', 'M', '000000'))
df_pin = df_pin.withColumn('save_location', regexp_replace('save_location', 'Local save in /', ''))
df_pin = df_pin.withColumn("follower_count",  
                                  df_pin["follower_count"] 
                                  .cast('int')) 
df_pin = df_pin.withColumn("downloaded",  
                                  df_pin["downloaded"] 
                                  .cast('int')) 
df_pin = df_pin.select("ind","unique_id","title","description","follower_count","poster_name","tag_list","is_image_or_video","image_src","save_location","category")
df_pin = df_pin.na.fill("Nones")

In [0]:
#SQL QUERIES
df_pin.createOrReplaceTempView("pin")
df_user.createOrReplaceTempView("user")
df_geo.createOrReplaceTempView("geo")

#Most popular Pinterest category people post to based on their country

df_popular_categories_by_country = spark.sql("SELECT country, category, COUNT(category) AS category_count FROM geo INNER JOIN pin ON pin.ind = geo.ind GROUP BY country,category ORDER BY category_count DESC")
df_popular_categories_by_country.show()

+-------------------+--------------+--------------+
 country| category|category_count|
+-------------------+--------------+--------------+
Antigua and Barbuda| christmas| 2|
 Algeria| quotes| 2|
 American Samoa| beauty| 2|
 Isle of Man| art| 2|
 Djibouti|diy-and-crafts| 1|
 Lithuania| education| 1|
 Chile| quotes| 1|
 Aruba| christmas| 1|
 Guatemala| art| 1|
 Bhutan|diy-and-crafts| 1|
 Ecuador| mens-fashion| 1|
 Bahamas| travel| 1|
 American Samoa| tattoos| 1|
 Algeria| christmas| 1|
 Canada| travel| 1|
 Argentina|diy-and-crafts| 1|
 Christmas Island| travel| 1|
 Monaco| art| 1|
 Seychelles| beauty| 1|
 Austria| tattoos| 1|
+-------------------+--------------+--------------+
only showing top 20 rows

In [0]:
#Most popular category per year between 2018 and 2022

df_popular_categories_per_year_between_2018_and_2022 = spark.sql("SELECT YEAR(timestamp) AS year, category, COUNT(category) AS category_count FROM geo INNER JOIN pin ON pin.ind = geo.ind WHERE YEAR(timestamp) < 2023 and YEAR(timestamp) > 2017 GROUP BY YEAR(timestamp),category ORDER BY category_count DESC LIMIT 12")
df_popular_categories_per_year_between_2018_and_2022.show()

+----+--------------+--------------+
year| category|category_count|
+----+--------------+--------------+
2019|diy-and-crafts| 4|
2018| art| 3|
2020| christmas| 3|
2022| quotes| 3|
2022| art| 3|
2018| education| 3|
2018| travel| 2|
2020| beauty| 2|
2020| home-decor| 2|
2019| finance| 2|
2021| quotes| 2|
2018| quotes| 2|
+----+--------------+--------------+

In [0]:
#User with most followers per country

df_user_with_most_followers_per_country = spark.sql("SELECT country, user_name, follower_count FROM user INNER JOIN pin ON pin.ind = user.ind INNER JOIN geo ON geo.ind = pin.ind ORDER BY follower_count DESC")
df_user_with_most_followers_per_country.show()

+-------------------+-------------------+--------------+
 country| user_name|follower_count|
+-------------------+-------------------+--------------+
 American Samoa| Aaron Abbott| 5000000|
 American Samoa| Aaron Abbott| 5000000|
 Afghanistan|Alexandria Alvarado| 2000000|
Antigua and Barbuda| Ann Chung| 1000000|
 Andorra| Alison Bell| 1000000|
 Algeria| Aaron Abbott| 942000|
 Algeria| Aaron Abbott| 942000|
 Lesotho| Regina Morales| 908000|
 Aruba| Debra Becker| 874000|
 Andorra| Amanda Brown| 799000|
 Pitcairn Islands| Elizabeth Terry| 671000|
 Algeria| Angela Allen| 613000|
 Maldives| Daniel Roy| 330000|
 Algeria| Alexis Bennett| 326000|
 Monaco| Glenn Hammond| 227000|
 Comoros| Gary Barker| 196000|
 Andorra| Aaron Alexander| 190000|
 Andorra| Aaron Alexander| 190000|
 Cape Verde| Alexander Perez| 178000|
 Djibouti| Jesus Arellano| 130000|
+-------------------+-------------------+--------------+
only showing top 20 rows

In [0]:
#Country with the user with the most followers

df_most_followers_per_country = spark.sql("WITH user_followers AS (SELECT country, user_name, follower_count FROM user INNER JOIN pin ON pin.ind = user.ind INNER JOIN geo ON geo.ind = pin.ind ORDER BY follower_count DESC) SELECT country, MAX(follower_count) AS follower_count FROM user_followers GROUP BY country ORDER BY follower_count DESC LIMIT 1")
df_most_followers_per_country.show()

+--------------+--------------+
 country|follower_count|
+--------------+--------------+
American Samoa| 5000000|
+--------------+--------------+

In [0]:
#Most popular category for different age groups

df_most_popular_category_per_age_group = spark.sql("SELECT category, COUNT(category) AS category_count, CASE WHEN age BETWEEN 18 AND 24 THEN '18-24' WHEN age BETWEEN 25 AND 35 THEN '25-35' WHEN age BETWEEN 36 AND 51 THEN '36-50' ELSE '50+' END AS age_group FROM pin INNER JOIN user ON pin.ind = user.ind GROUP BY age_group,category ORDER BY category_count DESC")
df_most_popular_category_per_age_group.show()

+--------------+--------------+---------+
 category|category_count|age_group|
+--------------+--------------+---------+
 art| 5| 25-35|
 christmas| 4| 25-35|
 education| 3| 18-24|
diy-and-crafts| 3| 18-24|
 quotes| 3| 18-24|
 mens-fashion| 3| 18-24|
 art| 3| 36-50|
 beauty| 3| 18-24|
 tattoos| 3| 18-24|
 travel| 2| 36-50|
diy-and-crafts| 2| 36-50|
diy-and-crafts| 2| 25-35|
 quotes| 2| 36-50|
 travel| 2| 25-35|
 christmas| 2| 18-24|
 vehicles| 2| 36-50|
 beauty| 2| 50+|
 home-decor| 2| 36-50|
 art| 2| 18-24|
 finance| 2| 18-24|
+--------------+--------------+---------+
only showing top 20 rows

In [0]:
#Median follower_count for each age group

df_median_follower_count_per_age_group = spark.sql("SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count, CASE WHEN age BETWEEN 18 AND 24 THEN '18-24' WHEN age BETWEEN 25 AND 35 THEN '25-35' WHEN age BETWEEN 36 AND 51 THEN '36-50' ELSE '50+' END AS age_group FROM pin INNER JOIN user ON pin.ind = user.ind GROUP BY age_group ORDER BY median_follower_count DESC")
df_median_follower_count_per_age_group.show()

+---------------------+---------+
median_follower_count|age_group|
+---------------------+---------+
 190000.0| 18-24|
 41500.0| 25-35|
 9000.0| 50+|
 4500.0| 36-50|
+---------------------+---------+

In [0]:
#Amount of users joined per year

df_amount_of_users_joined_per_year = spark.sql("SELECT YEAR(timestamp) AS post_year, COUNT(YEAR(date_joined)) AS number_users_joined FROM user INNER JOIN geo ON geo.ind = user.ind WHERE YEAR(timestamp) BETWEEN 2015 AND 2022 GROUP BY post_year")
df_amount_of_users_joined_per_year.show()

+---------+-------------------+
post_year|number_users_joined|
+---------+-------------------+
 2018| 17|
 2022| 10|
 2019| 12|
 2020| 14|
 2017| 6|
 2021| 9|
+---------+-------------------+

In [0]:
#Median follower count based on joining year

df_median_follower_count_per_joining_year = spark.sql("SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count, YEAR(timestamp) AS post_year FROM pin INNER JOIN geo ON pin.ind = geo.ind WHERE YEAR(timestamp) BETWEEN 2015 AND 2020 GROUP BY post_year ORDER BY median_follower_count DESC")
df_median_follower_count_per_joining_year.show()

+---------------------+---------+
median_follower_count|post_year|
+---------------------+---------+
 469500.0| 2017|
 41500.0| 2020|
 26000.0| 2019|
 19000.0| 2018|
+---------------------+---------+

In [0]:
#Median follower count based on year and age group

df_median_follower_count_per_age_group_and_year = spark.sql("SELECT YEAR(timestamp) AS post_year, PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count, CASE WHEN age BETWEEN 18 AND 24 THEN '18-24' WHEN age BETWEEN 25 AND 35 THEN '25-35' WHEN age BETWEEN 36 AND 51 THEN '36-50' ELSE '50+' END AS age_group FROM pin INNER JOIN geo ON pin.ind = geo.ind INNER JOIN user ON geo.ind = user.ind WHERE YEAR(timestamp) BETWEEN 2015 AND 2020 GROUP BY age_group,post_year ORDER BY median_follower_count DESC")
df_median_follower_count_per_age_group_and_year.show()

+---------+---------------------+---------+
post_year|median_follower_count|age_group|
+---------+---------------------+---------+
 2017| 806500.0| 18-24|
 2017| 326000.0| 50+|
 2019| 60000.0| 18-24|
 2020| 47000.0| 18-24|
 2020| 47000.0| 25-35|
 2018| 27000.0| 25-35|
 2018| 19000.0| 36-50|
 2018| 14000.0| 18-24|
 2017| 6000.0| 25-35|
 2018| 5000.0| 50+|
 2020| 4657.0| 36-50|
 2019| 3500.0| 36-50|
 2019| 3000.0| 25-35|
 2020| 20.0| 50+|
+---------+---------------------+---------+